# What is a TensorFlow Session?

https://danijar.com/what-is-a-tensorflow-session/

* 그래프는 computation을 정의합니다. 그래프에서는 그 어떤 계산도 일어나지 않고 그 어떤 값도 가지지 않습니다. 그저 op들을 정의하는 것 뿐입니다.
* 세션은 그래프 전체 또는 일부를 실행합니다. 실행을 위해 리소스를 할당하고 중간 결과나 변수의 실제 값을 가집니다. 

In [2]:
import tensorflow as tf

### 그래프 정의하기

다음과 같이 하나의 변수와 3개의 op을 이용해 그래프를 정의해 보겠습니다. 

In [4]:
graph = tf.Graph()
with graph.as_default():
    variable = tf.Variable(42, name='foo') # 변수 정의
    initialize = tf.global_variables_initializer() # 초기화
    assign = variable.assign(13) # 할당

노트 : 사실 TF는 자동으로 디폴트 그래프를 생성해 줍니다. 따라서 두 줄은 일반적으로 생략해도 무방합니다. 아래에 나올 세션도 어떤 그래프의 핸들러 역할을 하라고 명시적으로 주어지지 않으면 디폴트 그래프에 대한 핸들어 역할을 해 주게 됩니다. 

### 세션에서 computation 실행하기

그래프에 만들어 둔 op을 실행하기 위해서는 이 그래프에 대한 세션을 생성해야 합니다. 이 세션은 그 변수를 저장하기 위해 메모리를 할당할 것입니다.

In [6]:
with tf.Session(graph=graph) as sess:
  sess.run(initialize)
  sess.run(assign)
  print sess.run(variable)

13


변수의 값은 하나의 세션 내에서만 유효합니다. 아래와 같이 다른 세션을 만들어서 이 변수의 값을 요구하면 어떻게 되는 지 봅시다. 

In [14]:
with tf.Session(graph=graph) as sess:
    print sess.run(variable) 
# error - Attempting to use uninitialized value foo

FailedPreconditionError: Attempting to use uninitialized value foo
	 [[Node: _send_foo_0 = _Send[T=DT_INT32, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=-4994254303369096473, tensor_name="foo:0", _device="/job:localhost/replica:0/task:0/cpu:0"](foo)]]

역시 초기화되지 않았기 때문에 에러가 발생합니다. 

다른 세션에서도 초기화만 하면 괜찮습니다. 하지만 새로운 세션에서의 값은 첫번째 세션에서의 값과 완전 독립적인 것이 됩니다. 

In [16]:
with tf.Session(graph=graph) as sess:
  sess.run(initialize)
  print sess.run(variable)

42
